In [ ]:
import matplotlib.pyplot as plt
import numpy as np

## Energy probes

probe_3 refers to 2^3 time steps per layer, probe_4 uses 2^4 time steps and so on

In [ ]:
# probe_4 = 

In [ ]:
# probe_5 = 

In [ ]:
# probe_6 = 

In [ ]:
probe_7 = {'timePerTimestep': 278.0430415517553, 'numLmtsBeingUsed': 14, 'numCoresActuallyOn': 1760, 'lmtActiveRatio': 0.96728703227056, 'numCores': 1753, 'numChipsActuallyBeingUsed': 16, 'power': {'idle': 1973.6666666666665, 'total': 1990.5659591753833, 'core': {'SRAM_static': 889.9635617860384, 'logic_static': 381.7943680062105, 'static': 1271.757929792249, 'dynamic': 11.095570315093466}, 'lakemont': {'SRAM_static': 2.6084613693420797, 'logic_static': 0.3990945895093382, 'static': 3.007555958851418, 'clock': 252.0, 'code': 5.80372219362336, 'dynamic': 257.80372219362334}, 'static': 1721.6666666666665, 'dynamic': 268.8992925087168}}

In [ ]:
probes = [probe_7] # [probe_4, probe_5, probe_6]
steps_per_sample = [1314]
first_output_spike = [1214]
classif_error = [26.86]

## Latencies in ms

In [ ]:
latencies = [probe['timePerTimestep']*steps_per_sample[i]/1000 for i, probe in enumerate(probes)]
print(latencies)

## Energy per sample in uJ 

In [ ]:
# 'decision' refers to the number of time steps until the average first output spike, which is shorter than overall amount of time steps per sample
dynamic_energy_cores_decision = [probe['timePerTimestep']*(probe['power']['core']['dynamic'])*first_output_spike[i]/1000 for i, probe in enumerate(probes)]
print(dynamic_energy_cores_decision, ": Dynamic energy on cores until the first output spike arrives on average")

total_energy_cores_decision = [probe['timePerTimestep']*(probe['power']['core']['dynamic']+probe['power']['core']['static'])*first_output_spike[i]/1000 for i, probe in enumerate(probes)]
print(total_energy_cores_decision, ": Total energy on cores until the first output spike arrives on average")

total_energy_decision = [probe['timePerTimestep']*(probe['power']['core']['dynamic']+probe['power']['core']['static']+probe['power']['lakemont']['static']+probe['power']['lakemont']['dynamic'])*first_output_spike[i]/1000 for i, probe in enumerate(probes)]
print(total_energy_decision, ": Total energy on cores and lakemonts until the first output spike arrives on average")

total_energy = [probe['timePerTimestep']*(probe['power']['core']['dynamic']+probe['power']['core']['static']+probe['power']['lakemont']['static']+probe['power']['lakemont']['dynamic'])*steps_per_sample[i]/1000 for i, probe in enumerate(probes)]
print(total_energy, ": Total energy on cores and lakemonts for all time steps per sample")

## EDP using different energy measurements

In [ ]:
edp_dynamic = [latency*energy/1000 for latency, energy in zip(latencies, dynamic_energy_cores_decision)] # dynamic energy of cores, time to decision
print(edp_dynamic)

# as calculated by TrueNorth
edp_truenorth = [latency*energy/1000 for latency, energy in zip(latencies, total_energy_decision)] # total energy cores + lakemont, time to decision
print(edp_truenorth)

# as calculated by Rueckauer et al. 2021
edp_bodo = [latency*energy/1000 for latency, energy in zip(latencies, total_energy)] # total energy cores + lakemont, all time steps
print(edp_bodo)

In [ ]:
#[100 - error for error in classif_error]
fig, ax1 = plt.subplots(1, 1)
ax1.set_axisbelow(True)
plt.grid(True,  which="both", axis='y')
plt.scatter(edp_bodo, classif_error, label='TTFS (ours)')
plt.plot(edp_bodo, classif_error)
plt.scatter(34926, 8.52, color='red', label='Rate (Rueckauer et al., 2021)')
# plt.ylim([0.5, 100])
plt.yscale('log')
plt.xlabel(r"Energy delay product [$\mu$Js]")
plt.ylabel(r"Classification accuracy error [%]")
plt.title("CIFAR10 SNN performance")
plt.legend()
plt.tight_layout()
plt.savefig("EDP-plot-cifar.png")

## Breakdown of power

In [ ]:
probe_of_choice = probe_4
probe_of_choice['power'].keys()
print("neuromorphic cores: {} static, {} dynamic.".format(probe_of_choice['power']['core']['static'], probe_of_choice['power']['core']['dynamic']))
print("x86 lakemont cores: {} static, {} dynamic.".format(probe_of_choice['power']['lakemont']['static'], probe_of_choice['power']['lakemont']['dynamic']))
print("sum of dynamic power: {}".format(probe_of_choice['power']['core']['dynamic'] + probe_of_choice['power']['lakemont']['dynamic']))
print("sum of static power: {}".format(probe_of_choice['power']['core']['static'] + probe_of_choice['power']['lakemont']['static']))
print("sum of core power: {}".format(probe_of_choice['power']['core']['static'] + probe_of_choice['power']['core']['dynamic']))
print("sum of x86 power: {}".format(probe_of_choice['power']['lakemont']['static'] + probe_of_choice['power']['lakemont']['dynamic']))
print("sum: {}".format(probe_of_choice['power']['lakemont']['static'] + probe_of_choice['power']['lakemont']['dynamic']+probe_of_choice['power']['core']['static'] + probe_of_choice['power']['core']['dynamic']))